In [ ]:
# Ejemplo 1:
# Modelo TROPN, obtención del arbol de alcanzabilidad factible en el tiempo,
# y obtención de ruta óptima para el modelo propuesto por Kim et al. (2015)


In [ ]:
import snakes.plugins
import tropn
snakes.plugins.load(tropn, "snakes.nets", "nets")
from nets import *
from tropn import *
from time import process_time

In [ ]:
n = PetriNet('TROPN-Kim-2015')
n.globals.declare("from tropn import process")

n.add_place(Place('p0',[], bound=0))
n.add_place(Place('p1',[], bound=1))
n.add_place(Place('p2',[], bound=1))
n.add_place(Place('p3',[], bound=1))
n.add_place(Place('p4',[], bound=1))
n.add_place(Place('p5',[], bound=1))
n.add_place(Place('p6',[], bound=1))
n.add_place(Place('p7',[], bound=1))
n.add_place(Place('p8',[], bound=1))
n.add_place(Place('p9',[], bound=0))


n.add_transition(Transition('t1', Expression('(x=="X1" or x=="X3") and dot=="dot"')))
n.add_transition(Transition('t2', Expression('(x=="X2") and dot=="dot"')))
n.add_transition(Transition('t3', Expression('(x=="X1_1") and dot=="dot"')))
n.add_transition(Transition('t4', Expression('(x=="X3_1") and dot=="dot"')))
n.add_transition(Transition('t5', Expression('(x=="X1_2" or x=="X2_1") and dot=="dot"')))
n.add_transition(Transition('t6', Expression('(x=="X1_2" or x=="X2_1" or x=="X3_1") and dot=="dot"')))
n.add_transition(Transition('t7', Expression('(x=="X1_3" or x=="X2_2" or x=="X3_2") and dot=="dot"')))
n.add_transition(Transition('t8', Expression('(x=="X1_4" or x=="X2_3" or x=="X3_3") and dot=="dot"')))
                                                        
for t in ['t1','t2','t3','t4','t5']:
    n.add_input('p6', t, MultiArc([Variable("dot")]))
    n.add_output('p6', t, MultiArc([Expression("dot")]))  

for t in ['t6','t7','t8']:
    n.add_input('p8', t, MultiArc([Variable("dot")]))
    n.add_output('p8', t, MultiArc([Expression("dot")]))  

n.add_input('p0', 't1', MultiArc([Variable('x')]))
n.add_output('p1', 't1', MultiArc([Expression('process(x)')]))
n.add_input('p0', 't2', MultiArc([Variable('x')]))
n.add_output('p2', 't2', MultiArc([Expression('process(x)')]))
n.add_input('p1', 't3', MultiArc([Variable('x')]))
n.add_output('p2', 't3', MultiArc([Expression('process(x)')]))
n.add_input('p1', 't4', MultiArc([Variable('x')]))
n.add_output('p7', 't4', MultiArc([Expression('x')]))
n.add_input('p2', 't5', MultiArc([Variable('x')]))
n.add_output('p7', 't5', MultiArc([Expression('x')]))
n.add_input('p7', 't6', MultiArc([Variable('x')]))
n.add_output('p3', 't6', MultiArc([Expression('process(x)')]))
n.add_input('p3', 't7', MultiArc([Variable('x')]))
n.add_output('p4', 't7', MultiArc([Expression('process(x)')]))
n.add_input('p4', 't8', MultiArc([Variable('x')]))
n.add_output('p9', 't8', MultiArc([Expression('x')]))

n.set_marking(Marking({'p0': MultiSet(["X1", "X2"]),
                      'p6': MultiSet(["dot"]),
                      'p8': MultiSet(["dot"])}))
                
M0 = n.get_marking()
M0

In [ ]:
# Los tiempos de espera se definen como diccionario
# Para cada token, basta señalar los tiempos de espera correspondientes
# en las plazas y transiciones pertinentes.
# Cualquier tiempo de espera que no esté declara explicitamente, será cero
n.delays = {'X1':{'t1':3},
        'X1_1':{'p1':5,'t3':2},
        'X1_2':{'p2':3,'t5':2,'p7':0,'t6':2},
        'X1_3':{'p3':5,'t7':2},
        'X1_4':{'p4':2,'t8':2,'p9':0},
          
        'X2':{'t2':3},
        'X2_1':{'p2':4,'t5':3,'p7':0,'t6':2},
        'X2_2':{'p3':4,'t7':1},
        'X2_3':{'p4':2,'t8':1,'p9':0},
          
        'X3':{'t1':2},
        'X3_1':{'p1':4,'t4':2,'p7':0,'t6':2},
        'X3_2':{'p3':4,'t7':1},
        'X3_3':{'p4':2,'t8':2,'p9':0}
        }


In [ ]:
# caso 1 en Kim
n.delays = {'X1':{'p0':0,'t1':3},
    'X1_1':{'p1':167,'t3':3},
    'X1_2':{'p2':52,'t5':3,'p7':0,'t6':3},
    'X1_3':{'p3':105,'t7':3},
    'X1_4':{'p4':287,'t8':3,'p9':0},
    'X2':{'p0':0,'t2':3},
    'X2_1':{'p2':145,'t5':3,'p7':0,'t6':3},
    'X2_2':{'p3':111,'t7':3},
    'X2_3':{'p4':172,'t8':3,'p9':0},
    'X3':{'p0':0,'t1':3},
    'X3_1':{'p1':124,'t4':3,'p7':0,'t6':3},
    'X3_2':{'p3':251,'t7':3},
    'X3_3':{'p4':174,'t8':3,'p9':0}
    }

In [ ]:
# El procedimiento general de simulación es el siguiente
time_start = process_time()
# 1. Definir el estado inicial a partir de un marcaje inicial, con
#    ningún proceso actualmente en ejecución
tokens = MultiSet(['X1', 'X2'])*2 + MultiSet(['X3'])*2
#tokens = MultiSet(['X1', 'X2'])
m = Marking({'p0': tokens, 'p6': MultiSet(['dot']), 'p8': MultiSet(['dot'])})
s0 = NetState(m,{},{})
set_state(n,s0)
# 2. Obtener el árbol de alcanzabilidad
markings, tree, optimal = time_feasible_reachability_tree(n,s0)
# ....................
time_elapsed = (process_time() - time_start)
print ("%5.3f secs" % (time_elapsed))
print (len(markings), "states found")
print (len(tree), "state transitions")

In [ ]:
# número de estados distintos
len(markings)

In [ ]:
# número de pasos en el árbol de alcanzabilidad factible en el tiempo
len(tree)

In [ ]:
# longitud de la ruta de mínimo tiempo de procesamiento
len(optimal)

In [ ]:
# la estructura de cada paso es:
# step[0]: marcaje inicial
# step[1]: marcaje final
# step[2]: transición disparada
# step[3]: tokens utilizados
# step[4]: tiempo que tardó en ejecutarse el paso
# step[5]: tiempo de procesamiento acumulado

for step in optimal:
    print (step[5], step[1])

In [ ]:
# save in file
f = open("kim_test_optimal.txt","w+")
for l in optimal:
    f.write(str(l))
    f.write("\r\n")
f.close()